In [1]:
import pickle
from gensim import corpora
import gensim
import os
from pyLDAvis_local import *
from pyLDAvis_local import gensim_local
import pandas as pd

from gensim import corpora
from gensim.models import LsiModel
from nltk.corpus import stopwords
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unidecode

In [2]:
def get_g1_text(topic):
    URL = 'https://g1.globo.com/{}/'.format(topic)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    
    titles = soup.find_all('a', {'class' : 'feed-post-link gui-color-primary gui-color-hover'})
    descs = soup.find_all('div', {'class' : 'feed-post-body-resumo'})

    noticias = []
    counter = 0

    for title, desc in zip(titles, descs):
        if counter < 5:
            noticias.append({
                'title' : unidecode.unidecode(title.text), 
                'desc' : unidecode.unidecode(desc.text),
                'link' : title['href']
                }
            )
        counter += 1

    return pd.DataFrame(noticias)

In [3]:
def concat_dataframes(df1, df2):
    frames = [df1,df2]
    df3 = pd.concat(frames).reset_index()
    del df3['index']
    
    return df3

In [4]:
concat_df = pd.DataFrame()

topics = ['economia/agronegocios', 'ciencia/', 'carnaval/2022/', 'df/distrito-federal/', 'fato-ou-fake/', 'economia/', 'politica/eleicoes/2022/', 'empreendedorismo/']

for topic in topics:
    df1 = get_g1_text(topic)
    
    df1['Title'] = topic
    
    df3 = concat_dataframes(df1, concat_df)
    concat_df = df3

In [5]:
df3.head()

,title,desc,link,Title
0,MEI por necessidade: conheca historias de quem...,Cerca de 6 milhoes pessoas se tornam MEIs desd...,https://g1.globo.com/empreendedorismo/noticia/...,empreendedorismo/
1,O que e empreendedorismo social? Empresarias e...,"Tania e Emika comandam a ""...com Lola"" e expli...",https://g1.globo.com/empreendedorismo/noticia/...,empreendedorismo/
2,Mais de 380 mil empresas aderiram ao programa ...,"Conhecido como Relp, o programa permite que e...",https://g1.globo.com/empreendedorismo/noticia/...,empreendedorismo/
3,TikTok para pequenos negocios: veja dicas para...,Plataforma vai muito alem das dancinhas que vi...,https://g1.globo.com/empreendedorismo/noticia/...,empreendedorismo/
4,Empresa de Pequeno Porte: como abrir o registr...,"Para se enquadrar como EPP, empresas tem limit...",https://g1.globo.com/empreendedorismo/noticia/...,empreendedorismo/


In [6]:
stop = stopwords.words('portuguese')

In [7]:
def stopwords_column(df, new_name,column):
    df[new_name] = df[column].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    return df

In [8]:
df3 = stopwords_column(df3, 'title_stopwords', 'title')
df3 = stopwords_column(df3, 'desc_stopwords', 'desc')

In [9]:
def make_cluster_graph(df, column, input_topics):
    texts = []

    for text in df3[column]:
        test = text.split()
        texts.append(test)
        
    text_dictionary = corpora.Dictionary(texts) 
    corpus = [text_dictionary.doc2bow(text) for text in texts]
    pickle.dump(corpus, open('corpus.pkl', 'wb'))
    text_dictionary.save('text_dictionary.gensim')
    
    topics = input_topics

    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = topics, id2word=text_dictionary, passes=15)
    ldamodel.save('model.gensim')
    topics = ldamodel.print_topics(num_words=6)

    for topic in topics:
        print(topic)
        
    text_dictionary = gensim.corpora.Dictionary.load('text_dictionary.gensim')
    corpus = pickle.load(open('corpus.pkl', 'rb'))
    lda = gensim.models.ldamodel.LdaModel.load('model.gensim')
    lda_display = gensim_local.prepare(lda, corpus, text_dictionary, sort_topics=False)
    
    return display(lda_display)

In [11]:
test_title = make_cluster_graph(df3, 'title_stopwords', 8)
test_title

(0, '0.016*"melhora" + 0.016*"mulheres" + 0.016*"Beiju" + 0.016*"colorido" + 0.016*"comunidade" + 0.016*"Bahia"')
(1, '0.035*"1o" + 0.024*"PIB" + 0.024*"trimestre" + 0.013*"#FAKE" + 0.013*"#FATO" + 0.013*"base"')
(2, '0.019*"Brasil" + 0.019*"mil" + 0.019*"medicinal" + 0.019*"abre" + 0.019*"vagas" + 0.019*"Cannabis"')
(3, '0.011*"beneficios" + 0.011*"veja" + 0.011*"Foto" + 0.011*"Franklin" + 0.011*"DNA" + 0.011*"fascinante"')
(4, '0.016*"#FAKE" + 0.016*"Brasileiro" + 0.016*"mil" + 0.016*"R$" + 0.016*"programa" + 0.016*"propria,"')
(5, '0.013*"#FAKE" + 0.013*"imagem" + 0.013*"O" + 0.013*"vermelhas" + 0.013*"mostre" + 0.013*"audio"')
(6, '0.027*"diz" + 0.018*"Lula" + 0.018*"O" + 0.018*"sobre" + 0.010*"erva" + 0.010*"uso"')
(7, '0.026*"carnavalesco" + 0.014*"contra" + 0.014*"oferta" + 0.014*"quarta-feira" + 0.014*"Ferreira" + 0.014*"Policia"')


C:\Users\55119\Desktop\insper\9 semestre\Analise De Textos\AnaliseTextos\pyLDAvis_local\_prepare.py:230: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \


In [12]:
test_desc = make_cluster_graph(df3, 'desc_stopwords', 8)
test_desc

(0, '0.017*"Segundo" + 0.009*"faturamento" + 0.009*"empresas" + 0.009*"informacoes" + 0.009*"mil" + 0.009*"dois"')
(1, '0.009*"Confira" + 0.009*"canal" + 0.009*"gravado" + 0.009*"Audio" + 0.009*"ex-diretor" + 0.009*"gravou"')
(2, '0.010*"nao" + 0.010*"nesta" + 0.010*"ate" + 0.010*"R$" + 0.010*"Cerca" + 0.005*"relacao"')
(3, '0.010*"nao" + 0.010*"entanto," + 0.010*"e," + 0.010*"primeiro" + 0.006*"animais" + 0.006*"Lula"')
(4, '0.011*"Na" + 0.011*"fazer" + 0.011*"cor" + 0.011*"beterraba," + 0.011*"materia-prima" + 0.011*"laranja"')
(5, '0.016*"Serie" + 0.011*"vao" + 0.011*"ha" + 0.011*"enredo" + 0.011*"ate" + 0.006*"sobre"')
(6, '0.019*"vai" + 0.019*"Lula" + 0.010*"pode" + 0.010*"alem" + 0.010*"empresas." + 0.010*"clientes"')
(7, '0.019*"milhoes" + 0.019*"MEIs" + 0.010*"ano." + 0.010*"2020" + 0.010*"so" + 0.010*"animais"')


C:\Users\55119\Desktop\insper\9 semestre\Analise De Textos\AnaliseTextos\pyLDAvis_local\_prepare.py:230: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \
